In [6]:
# We need to create a SQLite database
# I'm going to use a API, but you can use CSVs

# Load data into a PANDAS dataframe -> load into the database

#### Load Dependencies

In [7]:
# data science
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import scipy.stats as stats

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, text, inspect, func

#### Read in Data File

In [8]:
# Create a file path
filepath = "../Resources/scrubbed.csv"

# Read in the data. 
df = pd.read_csv(filepath)

df.head()

C:\Users\Acer\AppData\Local\Temp\ipykernel_5656\3153306518.py:5: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611


#### Data Cleaning

In [9]:
df = df.dropna(subset=['country', 'state'])

In [10]:
# Fill nulls with 'unknown'
df["shape"] = df["shape"].fillna('unknown')
df["comments"] = df["comments"].fillna('no comment')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 68121 entries, 0 to 80331
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   datetime              68121 non-null  object 
 1   city                  68121 non-null  object 
 2   state                 68121 non-null  object 
 3   country               68121 non-null  object 
 4   shape                 68121 non-null  object 
 5   duration (seconds)    68121 non-null  object 
 6   duration (hours/min)  68121 non-null  object 
 7   comments              68121 non-null  object 
 8   date posted           68121 non-null  object 
 9   latitude              68121 non-null  object 
 10  longitude             68121 non-null  float64
dtypes: float64(1), object(10)
memory usage: 6.2+ MB


In [11]:
# Create a pared down list of shapes
item_shape = ['hexagon', 'delta', 'round', 'crescent', 'pyramid', 'flare', 'dome', 'changed', 'cross', 'cone']
df = df[~df["shape"].isin(item_shape)]

In [12]:
df = df[df["country"] == 'us']
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64666 entries, 0 to 80331
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   datetime              64666 non-null  object 
 1   city                  64666 non-null  object 
 2   state                 64666 non-null  object 
 3   country               64666 non-null  object 
 4   shape                 64666 non-null  object 
 5   duration (seconds)    64666 non-null  object 
 6   duration (hours/min)  64666 non-null  object 
 7   comments              64666 non-null  object 
 8   date posted           64666 non-null  object 
 9   latitude              64666 non-null  object 
 10  longitude             64666 non-null  float64
dtypes: float64(1), object(10)
memory usage: 5.9+ MB


In [13]:
df['datetime'] = pd.to_datetime(df['datetime'], format='mixed', errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64666 entries, 0 to 80331
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   datetime              64183 non-null  datetime64[ns]
 1   city                  64666 non-null  object        
 2   state                 64666 non-null  object        
 3   country               64666 non-null  object        
 4   shape                 64666 non-null  object        
 5   duration (seconds)    64666 non-null  object        
 6   duration (hours/min)  64666 non-null  object        
 7   comments              64666 non-null  object        
 8   date posted           64666 non-null  object        
 9   latitude              64666 non-null  object        
 10  longitude             64666 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(9)
memory usage: 5.9+ MB


In [14]:
df = df.dropna(subset=['datetime'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64183 entries, 0 to 80331
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   datetime              64183 non-null  datetime64[ns]
 1   city                  64183 non-null  object        
 2   state                 64183 non-null  object        
 3   country               64183 non-null  object        
 4   shape                 64183 non-null  object        
 5   duration (seconds)    64183 non-null  object        
 6   duration (hours/min)  64183 non-null  object        
 7   comments              64183 non-null  object        
 8   date posted           64183 non-null  object        
 9   latitude              64183 non-null  object        
 10  longitude             64183 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(9)
memory usage: 5.9+ MB


In [15]:
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64183 entries, 0 to 80331
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   datetime              64183 non-null  datetime64[ns]
 1   city                  64183 non-null  object        
 2   state                 64183 non-null  object        
 3   country               64183 non-null  object        
 4   shape                 64183 non-null  object        
 5   duration (seconds)    64183 non-null  object        
 6   duration (hours/min)  64183 non-null  object        
 7   comments              64183 non-null  object        
 8   date posted           64183 non-null  object        
 9   latitude              64183 non-null  float64       
 10  longitude             64183 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(8)
memory usage: 5.9+ MB


In [16]:
df['duration (seconds)'] = pd.to_numeric(df['duration (seconds)'], errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64183 entries, 0 to 80331
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   datetime              64183 non-null  datetime64[ns]
 1   city                  64183 non-null  object        
 2   state                 64183 non-null  object        
 3   country               64183 non-null  object        
 4   shape                 64183 non-null  object        
 5   duration (seconds)    64181 non-null  float64       
 6   duration (hours/min)  64183 non-null  object        
 7   comments              64183 non-null  object        
 8   date posted           64183 non-null  object        
 9   latitude              64183 non-null  float64       
 10  longitude             64183 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(7)
memory usage: 5.9+ MB


In [17]:
df = df.dropna(subset=['duration (seconds)'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64181 entries, 0 to 80331
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   datetime              64181 non-null  datetime64[ns]
 1   city                  64181 non-null  object        
 2   state                 64181 non-null  object        
 3   country               64181 non-null  object        
 4   shape                 64181 non-null  object        
 5   duration (seconds)    64181 non-null  float64       
 6   duration (hours/min)  64181 non-null  object        
 7   comments              64181 non-null  object        
 8   date posted           64181 non-null  object        
 9   latitude              64181 non-null  float64       
 10  longitude             64181 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(7)
memory usage: 5.9+ MB


In [18]:
df = df[df['datetime'].dt.year >= 1994]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58560 entries, 59 to 80331
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   datetime              58560 non-null  datetime64[ns]
 1   city                  58560 non-null  object        
 2   state                 58560 non-null  object        
 3   country               58560 non-null  object        
 4   shape                 58560 non-null  object        
 5   duration (seconds)    58560 non-null  float64       
 6   duration (hours/min)  58560 non-null  object        
 7   comments              58560 non-null  object        
 8   date posted           58560 non-null  object        
 9   latitude              58560 non-null  float64       
 10  longitude             58560 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(7)
memory usage: 5.4+ MB


In [19]:
df.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
59,1994-10-10 18:30:00,burnt hills,ny,us,rectangle,120.0,2 minutes,Giant rectangular craft (resembling an upside-...,10/23/2013,42.909722,-73.895556
60,1994-10-10 22:00:00,pinebergen,ar,us,light,5.0,5 seconds,Round&#44 bright&#44 low flying object silentl...,2/18/2001,34.102778,-91.992222
62,1995-10-10 19:45:00,milwaukee,wi,us,unknown,120.0,2 min.,Man on Hwy 43 SW of Milwaukee sees large&#44 ...,11/2/1999,43.038889,-87.906389
63,1995-10-10 22:40:00,oakland,ca,us,unknown,60.0,1 minute,Woman repts. bright light in NW sky&#44 sudde...,11/2/1999,37.804444,-122.269722
64,1996-10-10 03:20:00,higginsville,mo,us,triangle,3.0,3sec,illuminated triangular craft&#44 flying at hig...,2/16/2000,39.072500,-93.716944


In [20]:
df["hour"] = df.datetime.dt.hour
df["month"] = df.datetime.dt.month
df["year"] = df.datetime.dt.year
df["dayofweek"] = df.datetime.dt.strftime('%A')

In [21]:
df = df.rename(columns={"duration (seconds)": "duration_seconds", "duration (hours/min)": "duration_hours_min", "date posted": "date_posted"})
df.head()

,datetime,city,state,country,shape,duration_seconds,duration_hours_min,comments,date_posted,latitude,longitude,hour,month,year,dayofweek
59,1994-10-10 18:30:00,burnt hills,ny,us,rectangle,120.0,2 minutes,Giant rectangular craft (resembling an upside-...,10/23/2013,42.909722,-73.895556,18,10,1994,Monday
60,1994-10-10 22:00:00,pinebergen,ar,us,light,5.0,5 seconds,Round&#44 bright&#44 low flying object silentl...,2/18/2001,34.102778,-91.992222,22,10,1994,Monday
62,1995-10-10 19:45:00,milwaukee,wi,us,unknown,120.0,2 min.,Man on Hwy 43 SW of Milwaukee sees large&#44 ...,11/2/1999,43.038889,-87.906389,19,10,1995,Tuesday
63,1995-10-10 22:40:00,oakland,ca,us,unknown,60.0,1 minute,Woman repts. bright light in NW sky&#44 sudde...,11/2/1999,37.804444,-122.269722,22,10,1995,Tuesday
64,1996-10-10 03:20:00,higginsville,mo,us,triangle,3.0,3sec,illuminated triangular craft&#44 flying at hig...,2/16/2000,39.072500,-93.716944,3,10,1996,Thursday


In [22]:
df.dayofweek.value_counts()

dayofweek
Saturday     10419
Sunday        8476
Friday        8473
Wednesday     8053
Thursday      8015
Tuesday       7849
Monday        7275
Name: count, dtype: int64

In [23]:
df.hour.value_counts()

hour
21    8828
22    8078
20    6545
23    5613
19    4636
18    2999
0     2891
1     2258
17    1901
2     1579
3     1390
5     1278
4     1146
16    1132
15     979
6      943
12     911
14     873
13     868
10     868
11     831
9      742
7      668
8      603
Name: count, dtype: int64

In [ ]:
def categorize_shape(shape):
    if shape in ['changing', 'other', 'unknown']:
        return 'Other'
    elif shape in ['chevron', 'diamond', 'triangle']:
        return 'Angular'
    elif shape in ['cigar', 'cylinder', 'rectangle']:
        return 'Rectangular'
    elif shape in ['circle', 'disk', 'sphere']:
        return 'Circular'
    elif shape in ['fireball', 'flash', 'light']:
        return 'Luminous'
    elif shape in ['egg', 'oval', 'teardrop']:
        return 'Elliptical'
    elif shape == 'formation':
        return 'Formation'
    else:
        return 'Uncategorized'
# Apply category based on shape function
df['category'] = df['shape'].apply(categorize_shape)
df.head()

#### Below is the remaining starter code

In [24]:
# Create engine using the `ufo.sqlite` database file
engine = create_engine("sqlite:///ufo.sqlite")

In [25]:
# write to the database
df.to_sql("ufo", con=engine, index=False, method=None, if_exists="replace")

58560

In [26]:
# INSPECT to confirm existence

# Create the inspector and connect it to the engine
inspector_gadget = inspect(engine)

# Collect the names of tables within the database
tables = inspector_gadget.get_table_names()

# print metadata for each table
for table in tables:
    print(table)
    print("-----------")
    
    # get columns
    columns = inspector_gadget.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])

    print()

ufo
-----------
datetime DATETIME
city TEXT
state TEXT
country TEXT
shape TEXT
duration_seconds FLOAT
duration_hours_min TEXT
comments TEXT
date_posted TEXT
latitude FLOAT
longitude  FLOAT
hour INTEGER
month INTEGER
year INTEGER
dayofweek TEXT



In [27]:
# close the engine
engine.dispose()